<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de DataFrames de Spark con sparklyr</span></div>
<!-- v. 1.2 - Paulo Villegas, 2021 -->

Este notebook es una prueba con [sparklyr](http://spark.rstudio.com/) 

# Inicialización

Carga de la librería y creadión de la `SparkSession`

In [ ]:
# SPARK_HOME needs to be defined, or else sparklyr won't be able to launch a local instance
Sys.getenv('SPARK_HOME')

In [ ]:
# Load sparklyr. And also dplyr, which plays nicely with sparklyr
library(sparklyr)

library(dplyr)

In [ ]:
# Get a connection to a Spark local instance
sc <- spark_connect(master = "local")

In [ ]:
spark_version(sc)

# Lectura de datos

## Desde CSV

Leemos un [CSV](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.DataFrameReader)  para crear un `SparkDataFrame`

In [ ]:
# spark_read_csv es la función proporcionada por sparklyr para leer ficheros CSV y subirlos a Spark
?spark_read_csv

In [ ]:
uni2  <- spark_read_csv( sc, "uni_table", "../DATA/situacion-laboral-titulados-2019.csv", 
                         header=TRUE, options=list(comment="#") )

In [ ]:
class(uni2)

In [ ]:
colnames(uni2)

In [ ]:
head(uni2)

In [ ]:
# Como es una tabla pequeña, nos la podemos traer
collect(uni2)

## Desde tabla

_Nota: para que funcione el soporte a tablas en sparklyr, el backend SQL de Spark debe estar ajustado a `hive`. Esto se configura en `spark-defaults.conf` como_

    spark.sql.catalogImplementation=hive

In [ ]:
#situacion-laboral-titulados-2019
src_tbls(sc)

In [ ]:
uni3 <- tbl( sc, 'situacion_laboral_titulados_2019' )

In [ ]:
class(uni3)

# Manipulación de datos

Los verbos de `sparklyr` son los de `dplyr`, y por tanto también muy parecidos a los de `SparkR`. Aunque hay algunas diferencias en los argumentos enviados. Sparklyr es más "nativo" que SparkR.

## Agrupaciones
¿Cuántos licenciados hubo en 2014, por sexos?

In [ ]:
totales.por.sexos <- uni2 %>% 
    group_by( Sexo ) %>% 
    summarize( num=sum(Total) )

In [ ]:
collect(totales.por.sexos)

## Filtrado de casos

sparklyr sólo admite consultas con cláusulas explícitas. Para consultas SQL nativas se puede usar la interfaz DBI (ver más abajo)

In [ ]:
uni.f = filter( uni2, Tipo == "públicas")

In [ ]:
uni.f = filter( uni2, (Sexo == "mujeres") & (lower(Rama) == "ciencias") )

In [ ]:
uni.f = filter( uni2, substr(Rama,1,8) == "Ciencias" )

In [ ]:
# Esta va a fallar
uni.f = filter( uni2, 'Sexo = "mujeres" AND Rama = "Ciencias"' )

In [ ]:
# Cuando intentamos procesar el DataFrame se genera un error
head(uni.f)

## Selección y ordenación de columnas

In [ ]:
uni.trabajando = select( uni2, Rama, Total, Trabajando)

In [ ]:
head( uni.trabajando )

In [ ]:
uni.o = arrange( uni.trabajando, desc(Trabajando) )

In [ ]:
collect(uni.o)

## Creación de columnas

In [ ]:
uni.fraction = mutate( uni.trabajando, frac=Trabajando/Total)

In [ ]:
collect( uni.fraction )

In [ ]:
# Pintemos el resultado
library(ggplot2)
options(repr.plot.width = 10, repr.plot.height = 5)
qplot( Rama, frac, data=collect(uni.fraction) )

Agrupemos para tener las fracciones totales por rama de conocimiento ...

In [ ]:
uni.frac <- uni.trabajando %>% 
    group_by( Rama ) %>%
    summarize( Total=sum(Total), 
               Trabajando=sum(Trabajando) ) %>%
    mutate( frac=Trabajando/Total ) %>%
    arrange( desc(frac) ) %>%
    collect()

In [ ]:
uni.frac

## Consultas SQL
Para realizar consultas en SQL, podemos usar el paquete [DBI](http://r-dbi.github.io/DBI/) de R, ya que spqrklyr permite conectarlo al backend de Spark

_(de nuevo, es necesario que el backend SQL de Spark estñe fijado a `hive`, como se menciona más arriba)_

In [ ]:
library('DBI')

In [ ]:
dbListTables(sc)

In [ ]:
# O, lo que es lo mismo
dbGetQuery(sc, 'SHOW TABLES')

In [ ]:
dbGetQuery(sc, "SELECT * from situacion_laboral_titulados_2019 WHERE Sexo = 'hombres' and Total > 5000")

--------------------------------------------------

# Ejercicio

## Probabilidad de inactividad

<span style="color: red;">Pregunta:</span> *Si soy hombre y he estudiado en una universidad pública ¿en qué rama de conocimiento tengo más probabilidad de estar inactivo?*

Nota: calcularlo para todas las filas e inspeccionar visualmente es trampa. Tiene que ser un procedimiento que permita extraer ese dato programáticamente

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>